In [1]:
!pip install pyspark

  Using cached pyspark-3.2.1-py2.py3-none-any.whl
  Using cached py4j-0.10.9.3-py2.py3-none-any.whl (198 kB)


In [2]:
#import necessary packages
from pyspark.sql import SQLContext
from pyspark import SparkContext

#create spark context
SparkContext.getOrCreate()
sc = SparkContext.getOrCreate("accident")

#create sql context
sqlCtx = SQLContext(sc)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "d:\users\rezaz\pycharmprojects\bigdata_introduction\venv\lib\site-packages\py4j\clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\rezaz\AppData\Local\Programs\Python\Python39\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\users\rezaz\pycharmprojects\bigdata_introduction\venv\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "d:\users\rezaz\pycharmprojects\bigdata_introduction\venv\lib\site-packages\py4j\clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending 

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

Load the data from the csv files into DataFrames.

In [4]:
crashes = sqlCtx.read.option('header','true').csv('data/Traffic_Crashes_-_Crashes.csv')
vehicles = sqlCtx.read.option('header','true').csv('data/Traffic_Crashes_-_Vehicles.csv')
peoples = sqlCtx.read.option('header','true').csv('data/Traffic_Crashes_-_People.csv')
print(type(crashes))
peoples.dtypes


[]

Find the ratio of number of crashes where the person involved was using cell phone to that where the person was not using the cell phone.

In [1]:
peoples.createOrReplaceTempView("PERSON_DATA")
groupDF = sqlCtx.sql("SELECT DRIVER_ACTION, count(*) from PERSON_DATA group by DRIVER_ACTION")
groupDF.sort("count(1)","DRIVER_ACTION").show(30, truncate=True)

NameError: name 'peoples' is not defined

In [14]:
from pyspark.sql.types import StringType

phone = peoples.groupby("DRIVER_ACTION").count().filter((peoples.DRIVER_ACTION == 'CELL PHONE USE OTHER THAN TEXTING') | (peoples.DRIVER_ACTION == 'TEXTING'))

phone_crashes = phone.groupBy().sum('count').collect()[0][0]
print("crashes that occurs because of phone : ", phone_crashes)

no_phone = peoples.where((peoples.DRIVER_ACTION != "UNKNOWN") & (peoples.DRIVER_ACTION != "NONE"))\
.groupby("DRIVER_ACTION")\
.count()\
.filter((peoples.DRIVER_ACTION != 'CELL PHONE USE OTHER THAN TEXTING') & (peoples.DRIVER_ACTION != 'TEXTING'))

no_phone_crashes = no_phone.groupBy().sum('count').collect()[0][0]
print("crashes that occurs because of other : ", no_phone_crashes)

print("ratio: ", 100 * phone_crashes/no_phone_crashes)

crashes that occurs because of phone :  2125
crashes that occurs because of other :  409788
ratio:  0.5185608168125958


*bring in package to convert age code to int for sorting

In [6]:
from pyspark.sql.types import IntegerType

age = peoples.where((peoples.AGE > 0)).groupby(peoples.AGE.cast(IntegerType())).count().orderBy("count", ascending=False).limit(3)
age.show(120, truncate=False)


+----------------+-----+
|CAST(AGE AS INT)|count|
+----------------+-----+
|25              |26799|
|26              |26603|
|27              |26494|
+----------------+-----+



In [196]:
month = crashes.select((crashes.CRASH_MONTH).cast(IntegerType()))\
                    .groupby("CRASH_MONTH")\
                    .count()\
                    .orderBy("count", ascending=False).limit(1)
month.show(truncate=False)


+-----------+-----+
|CRASH_MONTH|count|
+-----------+-----+
|10         |56788|
+-----------+-----+



In [197]:
from pyspark.sql.types import IntegerType

df = crashes.select((crashes.CRASH_DAY_OF_WEEK).cast(IntegerType()))\
                    .groupby("CRASH_DAY_OF_WEEK")\
                    .count()\
                    .orderBy("count", ascending=True).limit(1)
df.show(truncate=False)

+-----------------+-----+
|CRASH_DAY_OF_WEEK|count|
+-----------------+-----+
|1                |72848|
+-----------------+-----+

